# Interannual Variability

In [ ]:
import huracanpy
import matplotlib.pyplot as plt

In [ ]:
# Load the data. Here: IBTrACS
data = huracanpy.load(source="ibtracs")

In [ ]:
# Get the basins
data["basin"] = huracanpy.info.basin(data.lon, data.lat)
# Equivalent to data = data.hrcn.add_basin()

In [ ]:
# Select the basin you're interested in (here: North Atlantic -- NATL)
## Option 1: Get only the points within this basin
NATL1 = data.where(data.basin == "NATL", drop=True)
## Option 2: Select tracks that have at least one point in this basin
### 2a : calling hurcanpy
NATL2a = huracanpy.trackswhere(data, data.track_id, lambda t: (t.basin == "NATL").any())
### 2b : Using the accessor
NATL2b = data.hrcn.trackswhere(lambda t: (t.basin == "NATL").any())

In [ ]:
# Plot interannual variability
fig, axs = plt.subplots(3, figsize=[15, 5], sharex=True)

## Frequency
gen = huracanpy.calc.gen_vals(
    NATL1, NATL1.time, NATL1.track_id
)  # eq. to gen = data.hrcn.get_gen_vals()
freq = gen.groupby("season").count().time
freq.plot(ax=axs[0], label="frequency")

## TC days
tcd = NATL1.groupby("season").apply(
    lambda s: (
        huracanpy.calc.track_duration(
            s.time, s.track_id
        ).sum()  # eq. to s.hrcn.get_track_duration().sum()
        / 24
    )  # Conversion because track_duration is in h
)
tcd.plot(ax=axs[1], label="TC days")

## ACE
ace = NATL1.groupby("season").apply(
    lambda s: huracanpy.tc.ace(s.wind).sum()  # eq. to s.hrcn.get_ace().sum()
)
ace.plot(ax=axs[2], label="ACE")

# Labels
axs[0].set_ylabel("#TC")
axs[1].set_ylabel("TC Days")
axs[2].set_ylabel("ACE")
for ax in axs:
    ax.set_xlabel("")
    ax.set_xticks(range(1980, 2022, 5))
    ax.grid()